# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle
#
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
            self.importance_G_function = self.__importance_gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
            self.importance_G_function = self.__importance_entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
            self.importance_G_function = self.__importance_misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')

        return 1 - np.sum((l_c ** 2 / l_s) + (r_c ** 2 / r_s), axis=1)

    def __entropy(self, l_c, l_s, r_c, r_s):
        l-np.sum((l_c * np.log(l_c / l_s)) + (r_c * np.log(r_c / r_s)),
                 axis=1).reshape(-1, 1) / (l_s + r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s + r_s).ravel()

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)

        return feature_ids[:int(np.sqrt(n_feature))]  # Ваш код в 1 строчку

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)

        return feature_ids[:int(np.log2(n_feature))]  # Ваш код в 1 строчку

    def __get_feature_ids_N(self, n_feature):
        return range(n_feature)  # Ваш код

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # sorted_x - отсортированный x,
        # sorted_x - значения y, соответствующие отсортированным x из sorted_x,
        sorted_x, sorted_y = self.__sort_samples(x, y)

        # Что делает этот блок кода?
        # Исключаются минимальные и максимальные значения
        # Проверка того, что в каждом классе окажется как минимум
        # min_sample_split элементов
        # Определяются индексы, для которых номер класса отличается от номера
        # класса соседнего элемента
        splitted_sorted_y = sorted_y[self.min_samples_split:-
                                     self.min_samples_split]
        r_border_ids = np.where(
            splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] \
            + (self.min_samples_split + 1)

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        # eq_el_count - число подряд идущих одинаковых эелементов
        # (за исключением последней группы)
        # one_hot_code - матрица принадлжености группы элементов
        # (отрезка) к классу
        # one_hot_code[i, j] == 1, если i-му отрезку соответствует класс j
        # class_increments[i, j] == n, если i-й отрезок содержит n элементов
        # класса j
        eq_el_count = r_border_ids - \
            np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros(
            (r_border_ids.shape[0], self.num_class))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(
            sorted_y[:self.min_samples_split], minlength=self.num_class)

        # Что делает этот блок кода?
        # l_class_count - число элементов каждого класса,
        # которое попадет в левую часть разбиения
        # r_class_count - число элементов каждого класса,
        # которое попадет в правую часть разбиения
        # l_sizes - мощность левого класса разбиения
        # r_sizes - мощность правого класса разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(
            y, minlength=self.num_class) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # Вычисляем impurity measure для каждого разбиения
        # Находим разбиение, для которого impurity measure->min
        # (т.е. граница минимизирует impurity measure)
        # (оптимальное разбиение)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # left_el_id - id элемента, с которого начинается
        # правый класс разбиения
        # Возвращаем impurity measure и усредненное значение threshold
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id - 1] + sorted_x[left_el_id]) / 2.0

    def __importance_gini(self, y):
        _, cnt = np.unique(y, return_counts=True)
        return len(y) * (1 - np.sum((cnt / len(y)) ** 2))

    def __importance_entropy(self, y):
        _, cnt = np.unique(y, return_counts=True)
        return -len(y) * np.sum((cnt / len(y)) * np.log2(counts / len(y)))

    def __importance_misclass(self, y):
        _, cnt = np.unique(y, return_counts=True)
        return len(y) * (1 - np.max(cnt / len(y)))

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        class_cnt = np.bincount(y)

        if (self.max_depth is not None) and (depth >= self.max_depth) or \
                np.max(class_cnt / y.shape[0]) >= self.sufficient_share or \
                x.shape[0] <= 2 * self.min_samples_split + 1:
            self.tree[node_id] = (self.LEAF_TYPE, class_cnt.argmax())
            return

        feature_ids = self.get_feature_ids(x.shape[1])

        thresholds = np.empty(len(feature_ids))

        gs = np.empty(len(feature_ids))

        for i in feature_ids:
            gs[i], thresholds[i] = self.__find_threshold(x[:, i], y)

        best_feature = gs.argmin()
        threshold = thresholds[best_feature]

        if threshold is None:
            self.tree[node_id] = (self.LEAF_TYPE, class_cnt.argmax())
            return

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_feature, threshold)

        if (x_l.shape[0] == 0) or (x_r.shape[0] == 0):
            self.tree[node_id] = (self.LEAF_TYPE, class_cnt.argmax())
            return

        self.tree[node_id] = (self.NON_LEAF_TYPE, best_feature, threshold)

        feature = np.argmin(gs)
        self.feature_importances_[feature] += self.importance_G_function(y) - \
            self.importance_G_function(y_l) - self.importance_G_function(y_r)

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1)

    def fit(self, x, y):
        # self.num_class - общее число классов
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1])
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(
    wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 4.38 ms, sys: 17 µs, total: 4.4 ms
Wall time: 3.56 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 54.8 ms, sys: 309 µs, total: 55.1 ms
Wall time: 97.3 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.7395197395197396

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.8407407407407407

## Подготовка данных Speed Dating Data 

In [21]:
#%%pycodestyle

speed_dating_csv = 'speed-dating-experiment/Speed Dating Data.csv'

df = pd.read_csv(speed_dating_csv, encoding='cp1251')
df = df.iloc[:, :97]

columns = [
    'iid', 'gender', 'pid', 'match', 'int_corr', 'samerace', 'age',
    'field_cd', 'mn_sat', 'tuition', 'race', 'imprace', 'imprelig',
    'income', 'goal', 'date', 'go_out', 'career_c', 'exphappy',
    'amb1_1', 'amb2_1', 'amb3_1', 'amb4_1', 'amb5_1',
    'attr1_1', 'attr2_1', 'attr3_1', 'attr4_1', 'attr5_1',
    'fun1_1', 'fun2_1', 'fun3_1', 'fun4_1', 'fun5_1',
    'intel1_1', 'intel2_1', 'intel3_1', 'intel4_1', 'intel5_1',
    'shar1_1', 'shar2_1', 'shar4_1',
    'sinc1_1', 'sinc2_1', 'sinc3_1', 'sinc4_1', 'sinc5_1'
]

df = df[columns]

df = df.dropna(subset=['age', 'date', 'imprelig', 'imprace'])

for col in ['career_c', 'field_cd']:
    df.loc[:, col] = df.loc[:, col].fillna(0)

for col in ['mn_sat', 'tuition', 'income']:
    df.loc[:, col] = df.loc[:, col].str.replace(',', '').astype(np.float)
    df.loc[:, col] = df[col].fillna(-999)

sum_columns = ['attr', 'sinc', 'intel', 'fun', 'amb', 'shar']

for i in range(1, 3):
    col = [c+str(i)+'_1' for c in sum_columns]

    df.loc[:, 'sum'] = df.loc[:, col].sum(axis=1)

    df.loc[:, col] = 100 * (df.loc[:, col].T / df.loc[:, 'sum'].T).T

df = df.drop(['sum'], axis=1)

df_male = df.query('gender==1').drop_duplicates(subset=['iid', 'pid'])
df_male = df_male.drop(['gender'], axis=1)
df_male = df_male.dropna()

df_female = df.query('gender==0').drop_duplicates(subset=['iid'])
df_female = df_female.drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)
df_female = df_female.dropna()

df_female.columns = df_female.columns + '_f'
df_female = df_female.drop(['pid_f'], axis=1)

df_mf = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')

df_mf = df_mf.drop(['iid', 'pid'], axis=1)

X = df_mf.iloc[:, 1:].values
y = df_mf.iloc[:, 0].values

In [10]:
#%%pycodestyle
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [11]:
#%%pycodestyle
%time clf.fit(X_train, y_train)

CPU times: user 104 ms, sys: 32 µs, total: 104 ms
Wall time: 126 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
#%%pycodestyle
%time my_clf.fit(X_train, y_train)

CPU times: user 1.57 s, sys: 7.79 ms, total: 1.58 s
Wall time: 1.75 s


## Проверка качества работы на Speed Dating Data

In [13]:
#%%pycodestyle
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5867417135709818

In [14]:
#%%pycodestyle
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.49643507676294557

## Задание 3

In [15]:
#%%pycodestyle

pipeline = Pipeline([
    ('clf', DecisionTreeClassifier()),
])

pipeline.fit(X_train, y_train)

f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro')

0.6086734693877551

## Задание 4

In [16]:
#%%pycodestyle

DTC_model = DecisionTreeClassifier()
DTC_model.fit(X_train, y_train)

MY_DTC_model = MyDecisionTreeClassifier()
MY_DTC_model.fit(X_train, y_train)

features = df_mf.columns[1:]

DTC_data = pd.Series(index=features, data=DTC_model.feature_importances_)
MY_DTC_data = pd.Series(index=features, data=MY_DTC_model.feature_importances_)

print('DecisionTreeClassifier')
print(DTC_data.sort_values(ascending=False).head(10))

print('MyDecisionTreeClassifier')
print(MY_DTC_data.sort_values(ascending=False).head(10))


DecisionTreeClassifier
int_corr      0.064532
fun1_1        0.026860
tuition_f     0.026180
amb4_1        0.023423
sinc1_1       0.021604
intel4_1_f    0.021127
field_cd_f    0.021080
sinc4_1       0.020798
date          0.020526
attr5_1       0.020200
dtype: float64
MyDecisionTreeClassifier
go_out_f     0.003493
tuition_f    0.002473
mn_sat_f     0.002422
shar4_1      0.001766
sinc4_1      0.001690
amb5_1       0.001654
sinc3_1_f    0.001606
fun4_1_f     0.001570
amb4_1_f     0.001538
mn_sat       0.001394
dtype: float64


In [17]:
#%%pycodestyle

print(f1_score(y_pred=DTC_model.predict(X_test), y_true=y_test, average='macro'))
print(f1_score(y_pred=MY_DTC_model.predict(X_test), y_true=y_test, average='macro'))

0.5936224489795918
0.49643507676294557


## Задание 5

In [18]:
#%%pycodestyle

parameters = {
    "min_samples_split":range(2, 10),
    "max_depth":range(1,10),
    "criterion":["gini", "entropy"]
}

grid_search_cv = RandomizedSearchCV(RandomForestClassifier(
    n_estimators=10), parameters, n_iter=50)

grid_search_cv.fit(X_train, y_train)

print("Best parameters:", grid_search_cv.best_params_)

Best parameters: {'min_samples_split': 9, 'max_depth': 8, 'criterion': 'entropy'}
